# Processing OpenTransit Bus Location Data

The purpose of this notebook is to present an example of working with the provided bus location data. We'll take a subset of this data (pulled only from buses on routes 1 and 14), along with geographical information about the stops on each route, to find every instance in our data set of a bus arriving at a stop along its route.

In [2]:
import json
import requests

from datetime import datetime, timedelta, timezone
from geopy.distance import distance

import pandas as pd
import numpy as np

from typing import List, Union

# Load the Data

In [3]:
def load_json(filename = 'routes24h20181015v2.json'):
    with open(filename, 'r') as f:
        return json.load(f)

In [4]:
bus_locations = load_json('route_14_week_data.json')

# Get All Stop and Bus IDs

We want to filter the location data so we're left with every instance of a bus arriving at a bus stop. We'll extract the location data for every stop on a given route (the current implementation was designed with finding these instances on a given route for a short time period). We do the same for bus locations on a given route.

In [5]:
def produce_stops(data: list, route: str) -> pd.DataFrame:
    stops = pd.io.json.json_normalize(data,
                                      record_path=['stops']) \
            .rename(columns={'lat': 'LAT',
                             'lon': 'LON',
                             'sid': 'SID'}) \
            .reindex(['SID', 'LAT', 'LON'], axis='columns')
    
    # obtain stop directions
    stops['DID'] = stops['SID'].map({stop: direction['id']
                                     for direction in requests
                                                      .get(f"http://restbus.info/api/agencies/sf-muni/routes/{route}")
                                                      .json()['directions']
                                     for stop in direction['stops']})
    
    # remove stops that don't have an associated direction
    stops = stops.dropna(axis='index', subset=['DID'])
    request = requests.get(f"http://restbus.info/api/agencies/sf-muni/routes/{route}")

    # obtain stop ordinals
    stops['ORD'] = stops['SID'].map({stop_meta['id']: ordinal
                                     for ordinal, stop_meta
                                     in enumerate(request.json()['stops'])})
    
    return stops

def produce_buses(data: list) -> pd.DataFrame:
     return pd.io.json.json_normalize(data,
                                      record_path=['routeStates', 'vehicles'],
                                      meta=[['routeStates', 'vtime']]) \
            .rename(columns={'lat': 'LAT',
                             'lon': 'LON',
                             'vid': 'VID',
                             'did': 'DID',
                             'routeStates.vtime': 'TIME'}) \
            .reindex(['TIME', 'VID', 'LAT', 'LON', 'DID'], axis='columns')

# Find Stops

In order to extract the arrivals from the location data, we find local minima for the distance, over time, of a given bus from a given stop. If a bus gets close enough to a given stop, we consider that bus as arriving at that particular stop.

There are two implementations presented for computing the distance between a bus and a stop - one uses the `geopy` library to compute the geodesic distance, and the other uses the haversine formula to compute the distance. While the former is more accurate, it can be prohibitively slow to use on large portions of the data set. The latter is faster, but less accurate.

In [6]:
# haversine formula for calcuating distance between two coordinates in lat lon
# from bird eye view; seems to be +- 8 meters difference from geopy distance
def haver_distance(latstop,lonstop,latbus,lonbus):

    latstop,lonstop,latbus,lonbus = map(np.deg2rad,[latstop,lonstop,latbus,lonbus])
    eradius = 6371000
    
    latdiff = (latbus-latstop)
    londiff = (lonbus-lonstop)
    
    a = np.sin(latdiff/2)**2 + np.cos(latstop)*np.cos(latbus)*np.sin(londiff/2)**2
    c = 2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    
    distance = eradius*c
    return distance

def find_eclipses(buses, stop):
    """
    Find movement of buses relative to the stop, in distance as a function of time.
    """
    def split_eclipses(eclipses, threshold=30*60*1000) -> List[pd.DataFrame]:
        """
        Split buses' movements when they return to a stop after completing the route.
        """
        disjoint_eclipses = []
        for bus_id in eclipses['VID'].unique(): # list of unique VID's
            # obtain distance data for this one bus
            bus = eclipses[eclipses['VID'] == bus_id].sort_values('TIME')
            #pprint.pprint(bus)
            #pprint.pprint(bus['TIME'].shift())
            #pprint.pprint(bus['TIME'].shift() + threshold)
            #print('===============')
            # split data into groups when there is at least a `threshold`-ms gap between data points
            group_ids = (bus['TIME'] > (bus['TIME'].shift() + threshold)).cumsum()

            # store groups
            for _, group in bus.groupby(group_ids):
                disjoint_eclipses.append(group)
        return disjoint_eclipses

    eclipses = buses.copy()
    #eclipses['DIST'] = eclipses.apply(lambda row: distance(stop[['LAT','LON']],row[['LAT','LON']]).meters,axis=1)
    
    stopcord = stop[['LAT', 'LON']]
    buscord = eclipses[['LAT', 'LON']]

    # calculate distances fast with haversine function 
    eclipses['DIST'] = haver_distance(stopcord['LAT'],stopcord['LON'],buscord['LAT'],buscord['LON'])
    # only keep positions within 750 meters within the given stop; (filtering out)
    eclipses = eclipses[eclipses['DIST'] < 750]
    
    # update the coordinates list 
    stopcord = stop[['LAT', 'LON']].values
    buscord = eclipses[['LAT', 'LON']].values
    
    # calculate distances again using geopy for the distance<750m values, because geopy is probably more accurate
    dfromstop = []
    for row in buscord:
        busdistance = distance(stopcord,row).meters
        dfromstop.append(busdistance)
    eclipses['DIST'] = dfromstop
    
    # for haversine function:
    #stopcord = stop[['LAT', 'LON']]
    #buscord = eclipses[['LAT', 'LON']]
    #eclipses['DIST'] = haver_distance(stopcord['LAT'],stopcord['LON'],buscord['LAT'],buscord['LON'])
    
    eclipses['TIME'] = eclipses['TIME'].astype(np.int64)
    eclipses = eclipses[['TIME', 'VID', 'DIST']]
    
    eclipses = split_eclipses(eclipses)
    
    return eclipses

def find_nadirs(eclipses):
    """
    Find points where buses are considered to have encountered the stop.
    
    Nadir is an astronomical term that describes the lowest point reached by an orbiting body.
    """
    def calc_nadir(eclipse: pd.DataFrame) -> Union[pd.Series, None]:
        nadir = eclipse.iloc[eclipse['DIST'].values.argmin()]
        if nadir['DIST'] < 100:  # if min dist < 100, then reasonable candidate for nadir
            return nadir
        else:  # otherwise, hardcore datasci is needed
            rev_eclipse = eclipse.iloc[::-1]
            rev_nadir = rev_eclipse.iloc[rev_eclipse['DIST'].values.argmin()]
            if nadir['TIME'] == rev_nadir['TIME']:  # if eclipse has a global min
                return nadir  # then it's the best candidate for nadir
            else:  # if eclipse's min occurs at two times
                mid_nadir = nadir.copy()
                mid_nadir['DIST'] = (nadir['DIST'] + rev_nadir['DIST'])/2
                return mid_nadir  # take the midpoint of earliest and latest mins
    
    nadirs = []
    for eclipse in eclipses:
        nadirs.append(calc_nadir(eclipse)[['VID', 'TIME']])
        
    return pd.DataFrame(nadirs)

In [7]:
# get all stops from a given data set
# getting all stops at once from the entire set of location data might take prohibitively long (~3-4 hours)
def get_all_stops(data):
    bus_stops = pd.DataFrame(columns = ["VID", "TIME", "SID", "DID", "ROUTE"])
    
    for route in {ele['rid'] for ele in data}:
        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: Starting with {route}.")
        try:
            stop_ids = [stop['id']
                for stop
                in requests.get(f"http://restbus.info/api/agencies/sf-muni/routes/{route}").json()['stops']]
                 
            route_data = [ele for ele in data if ele['rid'] == route]

            for stop_id in stop_ids:
                try:
                    stops = produce_stops(route_data, route)
                except:
                    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: could not produce stops df for {stop_id} on route {route}. Skipping.")
                    break

                buses = produce_buses(route_data)

                stop = stops[stops['SID'] == stop_id].drop_duplicates().squeeze()

                try: 
                    buses = buses[buses['DID'] == stop['DID']]
                except ValueError as err: # accounts for stops with no associated direction
                    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: skipping buses for {stop_id} and {route} due to ValueError: {err}")
                    continue

                eclipses = find_eclipses(buses, stop)
                nadirs = find_nadirs(eclipses)

                try:
                    # TODO: refactor
                    nadirs["TIME"] = nadirs["TIME"].apply(lambda x: datetime.fromtimestamp(x//1000, timezone(timedelta(hours = -8))))
                    #nadirs['DATE'] = nadirs['TIME'].apply(lambda x: x.date())
                    #nadirs['TIME'] = nadirs['TIME'].apply(lambda x: x.time())
                    nadirs["SID"] = stop_id
                    nadirs["DID"] = stop["DID"]
                    nadirs["ROUTE"] = route
                    bus_stops = bus_stops.append(nadirs, sort = True)
                except:
                    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: could not produce stops df for {stop_id} on route {route}. Skipping.", end = "\r")              
        except KeyError:
            print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: KeyError at {route}!")
            continue
                      
    #if len(bus_stops) > 0:
    #    bus_stops['timestamp'] = bus_stops[['DATE', 'TIME']].apply(lambda x: datetime.strptime(f"{x['DATE'].isoformat()} {x['TIME'].isoformat()} -0800", 
    #                                                                                   "%Y-%m-%d %H:%M:%S %z"), axis = 'columns')
    
    return bus_stops

In [8]:
sample_stops = get_all_stops(bus_locations)

Thu Feb 28 09:47:17 PM: Starting with 14.
Thu Feb 28 10:05:42 PM: skipping buses for 37742 and 14 due to ValueError: Can only compare identically-labeled Series objects
Thu Feb 28 10:07:16 PM: skipping buses for 7742 and 14 due to ValueError: Can only compare identically-labeled Series objects


`get_all_stops` produces a DataFrame containing every instance of a bus arriving at a stop from the given data. Columns of note:

- *DID*: the direction ID of the bus; the O/I in the string is the most relevant part of this value, as it indicates whether the bus is outbound or inbound.

- *SID*: the ID of the stop at which the bus arrived.

- *VID*: the vehicle ID of the bus.

In [9]:
sample_stops.sample(10)

,DID,ROUTE,SID,TIME,VID
121084,14___O_F00,14,5598,2019-01-15 14:35:36-08:00,7244
241233,14___I_F00,14,5620,2019-01-17 04:43:43-08:00,7293
368542,14___O_F00,14,5548,2019-01-18 16:10:39-08:00,7229
249550,14___I_F00,14,5605,2019-01-17 07:05:04-08:00,7224
265166,14___O_F00,14,5616,2019-01-17 10:47:30-08:00,7277
489948,14___O_F00,14,5607,2019-01-20 15:29:30-08:00,7249
251234,14___I_F00,14,5537,2019-01-17 07:30:06-08:00,7224
237039,14___O_F00,14,5539,2019-01-17 00:59:02-08:00,5433
357710,14___O_F00,14,5562,2019-01-18 13:42:32-08:00,7268
366753,14___I_F00,14,5583,2019-01-18 15:48:08-08:00,7226


In [10]:
sample_stops.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69564 entries, 45 to 509886
Data columns (total 5 columns):
DID      69564 non-null object
ROUTE    69564 non-null object
SID      69564 non-null object
TIME     69564 non-null object
VID      69564 non-null object
dtypes: object(5)
memory usage: 24.4 MB


In [11]:
with open('route_14_week_stops.json', 'w') as f:
    sample_stops.reset_index().drop(labels = 'index', axis = 'columns').to_json(f)